We are uploading all csv files of the Scarlet & Violet main and special sets.

Sets: SV01, SV02, SV03, SV: Scarlet & Violet 151, SV04, SV: Paldean Fates, SV05, SV06, SV: Shrouded Fable, SV07, SV08, SV: Prismatic Evolutions, SV09

In [15]:
import pandas as pd
import scipy
sv01_df = pd.read_csv('SV01ScarletAndVioletBaseSetProductsAndPrices.csv')
sv02_df = pd.read_csv('SV02PaldeaEvolvedProductsAndPrices.csv')
sv03_df = pd.read_csv('SV03ObsidianFlamesProductsAndPrices.csv')
sv04_df = pd.read_csv('SV04ParadoxRiftProductsAndPrices.csv')
sv05_df = pd.read_csv('SV05TemporalForcesProductsAndPrices.csv')
sv06_df = pd.read_csv('SV06TwilightMasqueradeProductsAndPrices.csv')
sv07_df = pd.read_csv('SV07StellarCrownProductsAndPrices.csv')
sv08_df = pd.read_csv('SV08SurgingSparksProductsAndPrices.csv')
sv09_df = pd.read_csv('SV09JourneyTogetherProductsAndPrices.csv')
sv_151_df = pd.read_csv('SVScarletAndViolet151ProductsAndPrices.csv')
sv_pf_df = pd.read_csv('SVPaldeanFatesProductsAndPrices.csv')
sv_sf_df = pd.read_csv('SVShroudedFableProductsAndPrices.csv')
sv_pe_df = pd.read_csv('SVPrismaticEvolutionsProductsAndPrices.csv')

We then merge all of these sets into a master set, filter out unnecessary columns, and add a column for the textual set name based on groupId variable.

In [18]:
master_df = pd.concat([sv01_df, sv02_df, sv03_df, sv04_df, sv05_df, sv06_df, sv07_df, sv08_df, sv09_df, sv_151_df, sv_pf_df, sv_sf_df, sv_pe_df])
master_df = master_df[['productId', 'name', 'groupId', 'marketPrice', 'extRarity']]

# filter for cards of specific rarity
master_df = master_df[(master_df['extRarity'] == 'Special Illustration Rare') | (master_df['extRarity'] == 'Illustration Rare') | (master_df['extRarity'] == 'Hyper Rare')]

# add textual set name column
def get_textset(setId):
    setId_list = [22873, 23120, 23228, 23286, 23381, 23473, 23537, 23651, 24073, 23237, 23353, 23529, 23821]
    textId_list = [
        'Scarlet And Violet Base',
        'Paldea Evolved',
        'Obsidian Flames',
        'Paradox Rift',
        'Temporal Forces',
        'Twilight Masquerade',
        'Stellar Crown',
        'Surging Sparks',
        'Journey Together',
        'Scarlet And Violet 151',
        'Paldean Fates',
        'Shrouded Fable',
        'Prismatic Evolutions'
    ]
    index = setId_list.index(setId)
    return textId_list[index]

master_df['set'] = master_df['groupId'].apply(get_textset)

# sort dataframe by card rarity
master_df = master_df.sort_values(by=['groupId', 'extRarity'])

# reset index
master_df = master_df.reset_index()
master_df.index +=1

display(master_df)

,index,productId,name,groupId,marketPrice,extRarity,set
1,437,490043,Miraidon ex - 253/198,22873,5.76,Hyper Rare,Scarlet And Violet Base
2,439,490045,Koraidon ex - 254/198,22873,5.50,Hyper Rare,Scarlet And Violet Base
3,483,490294,Basic Fighting Energy - 258/198,22873,3.14,Hyper Rare,Scarlet And Violet Base
4,484,490295,Basic Lightning Energy - 257/198,22873,5.01,Hyper Rare,Scarlet And Violet Base
5,485,490296,Rare Candy - 256/198,22873,7.83,Hyper Rare,Scarlet And Violet Base
...,...,...,...,...,...,...,...
437,338,623610,Iono's Bellibolt ex - 183/159,24073,136.62,Special Illustration Rare,Journey Together
438,339,623611,Lillie's Clefairy ex - 184/159,24073,328.63,Special Illustration Rare,Journey Together
439,340,623612,N's Zoroark ex - 185/159,24073,219.62,Special Illustration Rare,Journey Together
440,341,623613,Hop's Zacian ex - 186/159,24073,178.79,Special Illustration Rare,Journey Together
